In [1]:
#Importing the base level libraries
import pandas as pd
import numpy as np
import sklearn

In [4]:
#Importing required data
df= pd.read_csv('sky.csv')
print(df)

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,STAR,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,STAR,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,GALAXY,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,STAR,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,STAR,0.000590,3306,54922,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.237650e+18,131.316413,51.539547,18.81777,17.47053,16.91508,16.68305,16.50570,1345,301,3,161,5.033450e+17,GALAXY,0.027583,447,51877,246
9996,1.237650e+18,131.306083,51.671341,18.27255,17.43849,17.07692,16.71661,16.69897,1345,301,3,162,5.033400e+17,GALAXY,0.117772,447,51877,228
9997,1.237650e+18,131.552562,51.666986,18.75818,17.77784,17.51872,17.43302,17.42048,1345,301,3,162,8.222620e+18,STAR,-0.000402,7303,57013,622
9998,1.237650e+18,131.477151,51.753068,18.88287,17.91068,17.53152,17.36284,17.13988,1345,301,3,163,5.033400e+17,GALAXY,0.014019,447,51877,229


In [16]:
#Listing out the unique columns in the dataset
m= list(df.columns.unique())
m

['objid',
 'ra',
 'dec',
 'u',
 'g',
 'r',
 'i',
 'z',
 'run',
 'rerun',
 'camcol',
 'field',
 'specobjid',
 'class',
 'redshift',
 'plate',
 'mjd',
 'fiberid']

Converting the class column into required labels.

In [14]:
df['class'].unique()

array(['STAR', 'GALAXY', 'QSO'], dtype=object)

In [17]:
bool_map= dict(zip(['STAR', 'GALAXY', 'QSO'], range(0,3)))

In [18]:
df.dtypes

objid        float64
ra           float64
dec          float64
u            float64
g            float64
r            float64
i            float64
z            float64
run            int64
rerun          int64
camcol         int64
field          int64
specobjid    float64
class         object
redshift     float64
plate          int64
mjd            int64
fiberid        int64
dtype: object

In [21]:
for i in zip(df.dtypes.index, df.dtypes):
  if str(i[0])=='class':
    df[i[0]]= df[i[0]].map(bool_map)

In [22]:
df.head(n=3)

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,0,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,0,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,1,0.123111,287,52023,513


In [31]:
#Checking for null values
df.isnull().any()

objid        False
ra           False
dec          False
u            False
g            False
r            False
i            False
z            False
run          False
rerun        False
camcol       False
field        False
specobjid    False
class        False
redshift     False
plate        False
mjd          False
fiberid      False
dtype: bool

In [37]:
#Assigning vaiables
y = df['class'] 
X = df.drop(columns=['class'])

In [38]:
X

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,0.000590,3306,54922,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.237650e+18,131.316413,51.539547,18.81777,17.47053,16.91508,16.68305,16.50570,1345,301,3,161,5.033450e+17,0.027583,447,51877,246
9996,1.237650e+18,131.306083,51.671341,18.27255,17.43849,17.07692,16.71661,16.69897,1345,301,3,162,5.033400e+17,0.117772,447,51877,228
9997,1.237650e+18,131.552562,51.666986,18.75818,17.77784,17.51872,17.43302,17.42048,1345,301,3,162,8.222620e+18,-0.000402,7303,57013,622
9998,1.237650e+18,131.477151,51.753068,18.88287,17.91068,17.53152,17.36284,17.13988,1345,301,3,163,5.033400e+17,0.014019,447,51877,229


In [39]:
print(X.shape)
print(y.shape)

(10000, 17)
(10000,)


Let us first see how a simple decision tree classifier works on this dataset!

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [43]:
dt = DecisionTreeClassifier()

In [44]:
baseline_score = cross_val_score(dt, X, y, cv = 10, n_jobs = -1) 
print(baseline_score)
print("-----------------------------------------------------------")
print("Mean is {0: .3f} with std dev of {1: .3f} [Baseline]".format(baseline_score.mean(), baseline_score.std()))
    

[0.986 0.978 0.987 0.98  0.992 0.986 0.977 0.976 0.991 0.978]
-----------------------------------------------------------
Mean is  0.983 with std dev of  0.006 [Baseline]


In [ ]:
Bagging ( Bootstrapping + Aggregation )

#Bagging ( Bootstrapping + Aggregation )

In [46]:
#Importing necessary libraries for Bagging
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC

In [47]:
seed = 1075
np.random.seed(seed)

In [50]:
rf= RandomForestClassifier()
et= ExtraTreesClassifier()
knn= KNeighborsClassifier()
svc= SVC()

In [51]:
clf_array = [rf, et, knn, svc]

In [53]:
for clf in clf_array:
  without_bagging_scores= cross_val_score(clf, X, y, cv=10 ,n_jobs=-1 )
  bagging_clf= BaggingClassifier(clf, max_samples = 0.4, random_state = seed)
  bagging_scores = cross_val_score(bagging_clf, X, y, cv = 10, n_jobs = -1)
  print("Mean is {1: .3f} with std dev of {2: .3f} [{0}]".format(clf.__class__.__name__, 
                                                                     without_bagging_scores.mean(), without_bagging_scores.std()))
  print("Mean is {1: .3f} with std dev of {2: .3f} [Bagging {0}]".format(clf.__class__.__name__, 
                                                                     bagging_scores.mean(), bagging_scores.std()))
  print("=============================================================")

Mean is  0.989 with std dev of  0.003 [RandomForestClassifier]
Mean is  0.988 with std dev of  0.004 [Bagging RandomForestClassifier]
Mean is  0.978 with std dev of  0.008 [ExtraTreesClassifier]
Mean is  0.977 with std dev of  0.008 [Bagging ExtraTreesClassifier]
Mean is  0.742 with std dev of  0.032 [KNeighborsClassifier]
Mean is  0.777 with std dev of  0.034 [Bagging KNeighborsClassifier]
Mean is  0.797 with std dev of  0.032 [SVC]
Mean is  0.797 with std dev of  0.032 [Bagging SVC]


Here, we see that in the case of RandomForest and ExtraTrees, the sole classifiers are doing a much better job on the cross validation set. However, in the KNNs, the bagging overtakes in terms of accuracy.

# **Boosting**

In [54]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from mlxtend.classifier import EnsembleVoteClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [57]:
ada_boost= AdaBoostClassifier()
xgb_boost= XGBClassifier()
grad_boost = GradientBoostingClassifier()

In [58]:
boost_array = [ada_boost, grad_boost, xgb_boost]

In [59]:
labels = ['Ada Boost', 'Grad Boost', 'XG Boost']

In [60]:
for clf, label in zip([ada_boost, grad_boost, xgb_boost], labels):
    scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print("Accuracy is {1: .3f} with std dev of {2: .3f} [{0}]".format(label, scores.mean(), scores.std()))

Accuracy is  0.871 with std dev of  0.069 [Ada Boost]
Accuracy is  0.990 with std dev of  0.003 [Grad Boost]
Accuracy is  0.990 with std dev of  0.003 [XG Boost]


Out of all options that were tried so far, the XG Boost algorithm works the best on the given problem at hand.

## Stacking

In [61]:
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier

In [62]:
clf1 = KNeighborsClassifier()
clf2 = RandomForestClassifier()
lr = LogisticRegression()
## Defining stacking classifier
sclf = StackingClassifier(classifiers=[clf1, clf2], 
                          meta_classifier=lr)

In [63]:
labels = ['KNN', 'Random Forest', 'Stacking Classifier']
clf_array = [clf1, clf2, sclf]

In [64]:
for clf, label in zip(clf_array, labels):
    scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print("Accuracy is {1: .3f} with std dev of {2: .3f} [{0}]".format(label, scores.mean(), scores.std()))

Accuracy is  0.742 with std dev of  0.032 [KNN]
Accuracy is  0.989 with std dev of  0.004 [Random Forest]
Accuracy is  0.989 with std dev of  0.004 [Stacking Classifier]


The RandomForest and the Stacking Classifier seem to work with the same efficiency and clearly much better than the KNN algoithm.